# Benchmark part

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "sd-legacy/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
import torch
import time
from tqdm import tqdm
import pynvml
import datetime
import os

# Initialize NVIDIA Management Library for temperature monitoring
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

# Set up timing variables
benchmark_duration = 10  # seconds (or 300 for 5 minutes)
image_count = 0
total_gpu_time = 0
temp_readings = []

# Get current timestamp and create log filename with absolute path
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
current_dir = os.getcwd()
log_filename = os.path.join(current_dir, f"benchmark_{timestamp}.txt")

# Print the log file path to verify
print(f"Will save log to: {log_filename}")

# Prompt for generation
prompt = "a photo of an astronaut riding a horse on mars"

# Try to create the log file with error handling
try:
    with open(log_filename, "w") as log:
        log.write(f"GPU Benchmark - {timestamp}\n")
        log.write(f"Device: {torch.cuda.get_device_name(0)}\n")
        log.write(f"Benchmark duration: {benchmark_duration} seconds\n")
        log.write(f"Prompt: {prompt}\n")
        log.write("-" * 50 + "\n\n")
        log.write("DETAILED LOG:\n")
    print(f"Successfully created log file")
except Exception as e:
    print(f"Error creating log file: {e}")
    # Try creating in home directory as fallback
    log_filename = os.path.expanduser(f"~/benchmark_{timestamp}.txt")
    print(f"Trying alternate location: {log_filename}")
    with open(log_filename, "w") as log:
        log.write(f"GPU Benchmark - {timestamp}\n")

# Start the benchmark
print(f"Starting benchmark for {benchmark_duration} seconds...")
start_time = time.time()
end_time = start_time + benchmark_duration

# Function to safely append to log file
def append_to_log(message):
    try:
        with open(log_filename, "a") as log:
            log.write(message)
    except Exception as e:
        print(f"Error writing to log: {e}")

# Run until time is up
with tqdm() as pbar:
    while time.time() < end_time:
        # Get GPU temperature and add to list
        current_temp = pynvml.nvmlDeviceGetTemperature(handle, pynvml.NVML_TEMPERATURE_GPU)
        temp_readings.append(current_temp)
        
        # CUDA timing events
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        
        # Synchronize before generation
        torch.cuda.synchronize()
        
        # Record start time
        start_event.record()
        
        # Generate image (but don't save it)
        image = pipe(prompt).images[0]
        
        # Record end time
        end_event.record()
        torch.cuda.synchronize()
        
        # Calculate GPU time
        gpu_time_ms = start_event.elapsed_time(end_event)
        total_gpu_time += gpu_time_ms
        
        # Log this iteration
        append_to_log(f"Image {image_count}: Time={time.time()-start_time:.2f}s, Temp={current_temp}°C, GenTime={gpu_time_ms:.2f}ms\n")
        
        # Update counter and progress
        image_count += 1
        pbar.update(1)
        pbar.set_description(f"Generated: {image_count} imgs | Current temp: {current_temp}°C")

# Get final temperature reading
final_temp = pynvml.nvmlDeviceGetTemperature(handle, pynvml.NVML_TEMPERATURE_GPU)
temp_readings.append(final_temp)

# Calculate results
elapsed = time.time() - start_time
avg_time_ms = total_gpu_time / image_count if image_count > 0 else 0
avg_temp = sum(temp_readings) / len(temp_readings)
max_temp = max(temp_readings)

# Create summary
summary = "\n" + "="*50 + "\n"
summary += "BENCHMARK SUMMARY:\n"
summary += f"Benchmark completed in {elapsed:.2f} seconds\n"
summary += f"Images generated: {image_count}\n"
summary += f"Images per second: {image_count/elapsed:.2f}\n"
summary += f"Average GPU time per image: {avg_time_ms:.2f} ms\n"
summary += f"Total GPU processing time: {total_gpu_time/1000:.2f} seconds\n"
summary += f"GPU utilization: {(total_gpu_time/1000)/elapsed*100:.1f}%\n"
summary += f"\nTemperature Statistics:\n"
summary += f"  Starting temperature: {temp_readings[0]}°C\n"
summary += f"  Ending temperature: {final_temp}°C\n"
summary += f"  Average temperature: {avg_temp:.1f}°C\n"
summary += f"  Maximum temperature: {max_temp}°C\n"
summary += f"  Temperature increase: {final_temp - temp_readings[0]}°C\n"
summary += "="*50

# Print summary to console
print(summary)

# Add summary to log file
append_to_log(summary)

# Clean up
pynvml.nvmlShutdown()

print(f"Log saved to {log_filename}")

# Double check if file exists
if os.path.exists(log_filename):
    print(f"Confirmed: log file exists at {log_filename}")
    print(f"File size: {os.path.getsize(log_filename)} bytes")
else:
    print(f"Warning: Could not find log file at {log_filename}")

In [ ]:
# Install the Supabase client if needed
# !pip install supabase requests

from supabase import create_client
import datetime
import platform
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get Supabase credentials from environment variables
supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")

# Verify credentials were loaded
if not supabase_url or not supabase_key:
    print("❌ Error: Supabase credentials not found in .env file")
    print("Make sure you have a .env file with SUPABASE_URL and SUPABASE_KEY")
else:
    print(f"✅ Supabase credentials loaded successfully")

# Convert country code to flag emoji (Unicode regional indicator symbols)
def country_code_to_flag(country_code):
    if len(country_code) != 2 or not country_code.isalpha():
        return "🏳️"  # White flag for unknown
    
    # Convert each letter to regional indicator symbol
    # A-Z: 0x41-0x5A -> regional indicators: 0x1F1E6-0x1F1FF
    return ''.join(chr(ord(c.upper()) - ord('A') + ord('🇦')) for c in country_code)

# Get country information
try:
    country_response = requests.get("https://ipinfo.io/json")
    country_code = country_response.json().get("country", "Unknown")
    
    # Convert country code to flag emoji
    flag_emoji = country_code_to_flag(country_code)
    
except Exception as e:
    print(f"Error getting country info: {e}")
    flag_emoji = "🏳️"  # White flag for unknown

# Prepare benchmark results for Supabase using data from the previous run
benchmark_results = {
    "created_at": datetime.datetime.now().isoformat(),
    "gpu_type": torch.cuda.get_device_name(0),
    "number_images_generated": image_count,
    "max_heat": int(max_temp),
    "avg_heat": int(avg_temp),
    "country": flag_emoji  # Store the flag emoji directly
}

# Print the data being uploaded
print("Uploading the following data to Supabase:")
for key, value in benchmark_results.items():
    print(f"  {key}: {value}")

# Upload to Supabase
try:
    print("\nConnecting to Supabase...")
    # Initialize Supabase client
    supabase = create_client(supabase_url, supabase_key)
    
    print("Uploading results...")
    # Insert benchmark results into the 'benchmark' table
    response = supabase.table('benchmark').insert(benchmark_results).execute()
    
    # Check for successful upload
    if hasattr(response, 'data') and response.data:
        print(f"✅ Results successfully uploaded to Supabase! {flag_emoji}")
        print(f"Inserted record ID: {response.data[0]['id'] if response.data and len(response.data) > 0 else 'unknown'}")
    else:
        print("❌ Error uploading to Supabase.")
        if hasattr(response, 'error'):
            print(f"Error details: {response.error}")
        
except Exception as e:
    print(f"❌ Error uploading to Supabase: {e}")
    print("\nTroubleshooting tips:")
    print("1. Check that your Supabase URL and API key are correct in the .env file")
    print("2. Verify your network connection")

In [ ]:
"""
i have all of the basics now i guess. next i need to make the python package 
"""